<a href="https://colab.research.google.com/github/iamukasa/hot_not/blob/master/hot_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import tensorflow as tf
from skimage import exposure
from tensorflow.contrib.layers import flatten
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.contrib.layers import flatten
import keras
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, ELU
from keras.layers.convolutional import Convolution2D, Cropping2D
from keras.layers.convolutional import Convolution2D as Conv2D
from keras.layers.pooling import MaxPooling2D
import cv2
import glob


Using TensorFlow backend.


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

!ls '/content/drive/My Drive/data'
!ls '/content/drive/My Drive/data/hot_dog'
!ls '/content/drive/My Drive/data/not_hot_dog'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
hot_dog  not_hot_dog
'hotdog (100).jpg'  'hotdog (213).jpg'	'hotdog (326).jpg'  'hotdog (439).jpg'
'hotdog (101).jpg'  'hotdog (214).jpg'	'hotdog (327).jpg'  'hotdog (43).jpg'
'hotdog (102).jpg'  'hotdog (215).jpg'	'hotdog (328).jpg'  'hotdog (440).jpg'
'hotdog (103).jpg'  'hotdog (216).jpg'	'hotdog (329).jpg'  'hotdog (441).jpg'
'hotdog (104).jpg'  'hotdog (217).jpg'	'hotdog (32).jpg'   'hotdog (442).jpg'
'hotdog (105).jpg'  'hotdog (218).jpg'	'hotdog (330).j

In [0]:
def rotateImage(img, angle):
    (rows, cols, ch) = img.shape
    M = cv2.getRotationMatrix2D((cols/2,rows/2), angle, 1)
    return cv2.warpAffine(img, M, (cols,rows))
    
    
def loadBlurImg(path, imgSize):
    img = cv2.imread(path)
    angle = np.random.randint(0, 360)
    img = rotateImage(img, angle)
    img = cv2.blur(img,(5,5))
    img = cv2.resize(img, imgSize)
    return img

def loadImgClass(classPath, classLable, classSize, imgSize):
    x = []
    y = []
    
    for path in classPath:
        img = loadBlurImg(path, imgSize)        
        x.append(img)
        y.append(classLable)
        
    return x, y


def loadData(img_size, classSize):
    hotdogs = glob.glob('/content/drive/My Drive/data/hot_dog/**/*.jpg', recursive=True)
    notHotdogs = glob.glob('/content/drive/My Drive/data/not_hot_dog/**/*.jpg', recursive=True)
    
    
    imgSize = (img_size, img_size)
    xHotdog, yHotdog = loadImgClass(hotdogs, 0, classSize, imgSize)
    xNotHotdog, yNotHotdog = loadImgClass(notHotdogs, 1, classSize, imgSize)
    print("There are", len(xHotdog), "hotdog images")
    print("There are", len(xNotHotdog), "not hotdog images")
    
    X = np.array(xHotdog + xNotHotdog)
    y = np.array(yHotdog + yNotHotdog)
    
    return X, y

In [0]:
def toGray(images):
    # rgb2gray converts RGB values to grayscale values by forming a weighted sum of the R, G, and B components:
    # 0.2989 * R + 0.5870 * G + 0.1140 * B 
    # source: https://www.mathworks.com/help/matlab/ref/rgb2gray.html
    
    images = 0.2989*images[:,:,:,0] + 0.5870*images[:,:,:,1] + 0.1140*images[:,:,:,2]
    return images

def normalizeImages(images):
    # use Histogram equalization to get a better range
    # source http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.equalize_hist
    images = (images / 255.).astype(np.float32)
    
    for i in range(images.shape[0]):
        images[i] = exposure.equalize_hist(images[i])
    
    images = images.reshape(images.shape + (1,)) 
    return images

def preprocessData(images):
    grayImages = toGray(images)
    #grayImages=images
    return normalizeImages(grayImages)

In [0]:
def karasModel(inputShape):
    model = Sequential()
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4),border_mode='valid', input_shape=inputShape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    model.add(Convolution2D(32, 5, 5))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    model.add(Convolution2D(32, 3, 3))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(240))

    model.add(Activation('relu'))
    model.add(Dense(120))

    #model.add(Activation('relu'))
    model.add(Dense(2))

    model.add(Activation('softmax'))
    return model 

  
def karasModel2(inputShape,n_layers):
#     model = Sequential()
    
#     model.add(Convolution2D(16, 8, 8, subsample=(4, 4),border_mode='valid', input_shape=inputShape))
#     model.add(ELU())
#     model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
#     model.add(ELU())
#     model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
#     model.add(Flatten())
#     model.add(Dropout(.2))
#     model.add(ELU())
#     model.add(Dense(512))
#     model.add(Dropout(.5))
#     model.add(ELU())
#     model.add(Dense(2))
#     model.add(Activation('softmax'))





# Define hyperparamters
    MIN_NEURONS = 20
    MAX_NEURONS = 120
    KERNEL = (3, 3)

    # Determine the # of neurons in each convolutional layer
    steps = np.floor(MAX_NEURONS / (n_layers + 1))
    nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
    nuerons = nuerons.astype(np.int32)

    # Define a model
    model = Sequential()

    # Add convolutional layers
    for i in range(0, n_layers):
        if i == 0:
            shape = (inputShape[0],inputShape[1], inputShape[2])
            model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
        else:
            model.add(Conv2D(nuerons[i], KERNEL))

        model.add(Activation('relu'))

    # Add max pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(MAX_NEURONS))
    model.add(Activation('relu'))

    # Add output layer
    model.add(Dense(2))
    model.add(Activation('sigmoid'))


    # Print a summary of the model
    model.summary()






    return model
  
  

In [0]:
size = 128
classSize = 400

scaled_X, y = loadData(size, classSize)
scaled_X = preprocessData(scaled_X)

n_classes = len(np.unique(y))

from keras.utils.np_utils import to_categorical
y = to_categorical(y)

There are 499 hotdog images
There are 498 not hotdog images


In [0]:
rand_staterand_sta  = np.random.randint(0, 100)

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_staterand_sta)

print("Number of classes =", n_classes)
print("train shape X", X_train.shape)
print("train shape y", y_train.shape)

Number of classes = 2
train shape X (797, 128, 128, 1)
train shape y (797, 2)


In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-10-04 09:14:07--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.45.111.123, 52.72.145.109, 52.45.248.161, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.45.111.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M   808KB/s    in 8.5s    

2018-10-04 09:14:16 (618 KB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
inputShape = (size, size, 1)
model = karasModel2(inputShape,4)


tbCallBack = keras.callbacks.hTensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)


model.compile('adam', 'binary_crossentropy', ['accuracy'])
history = model.fit(X_train, y_train, nb_epoch=10, validation_split=0.1,callbacks=[tbCallBack])





metrics = model.evaluate(X_test, y_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 20)      200       
_________________________________________________________________
activation_1 (Activation)    (None, 126, 126, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 44)      7964      
_________________________________________________________________
activation_2 (Activation)    (None, 124, 124, 44)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 122, 122, 68)      26996     
_________________________________________________________________
activation_3 (Activation)    (None, 122, 122, 68)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 120, 120, 92)      56396     
__________

/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 717 samples, validate on 80 samples
Epoch 1/10
717/717 [==============================] - 10s 14ms/step - loss: 0.8103 - acc: 0.5258 - val_loss: 0.7395 - val_acc: 0.3750
Epoch 2/10
717/717 [==============================] - 6s 9ms/step - loss: 0.6845 - acc: 0.5753 - val_loss: 0.6516 - val_acc: 0.7000
Epoch 3/10
717/717 [==============================] - 6s 9ms/step - loss: 0.6012 - acc: 0.6729 - val_loss: 0.7527 - val_acc: 0.5875
Epoch 4/10
717/717 [==============================] - 6s 9ms/step - loss: 0.4336 - acc: 0.8089 - val_loss: 0.8217 - val_acc: 0.6188
Epoch 5/10
717/717 [==============================] - 6s 9ms/step - loss: 0.2454 - acc: 0.9066 - val_loss: 1.1955 - val_acc: 0.5375
Epoch 6/10
717/717 [==============================] - 6s 9ms/step - loss: 0.0937 - acc: 0.9665 - val_loss: 2.0087 - val_acc: 0.5938
Epoch 7/10
717/717 [==============================] - 6s 9ms/step - loss: 0.1143 - acc: 0.9568 - val_loss: 1.8649 - val_acc: 0.6250
Epoch 8/10
717/717 [=========

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://0cc883ef.ngrok.io


In [0]:
model.save('model.h5')